<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/cdvae_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inverse Design of Next-Generation Superconductors Using Data-Driven Deep Generative Models

Reference: https://pubs.acs.org/doi/10.1021/acs.jpclett.3c01260

Author: Kamal Choudhary (kamal.choudhary@nist.gov)

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:15
🔁 Restarting kernel...


In [1]:
!conda --version

conda 23.11.0


In [2]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [3]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 75.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 

In [ ]:
# !pip install -q pytorch-lightning==1.3.6

In [4]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 253 (delta 38), reused 52 (delta 32), pack-reused 164
Receiving objects: 100% (253/253), 146.69 MiB | 17.36 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Updating files: 100% (99/99), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Running setup.py develop for cdvae


In [5]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [6]:
!echo $HYDRA_FULL_ERROR

1


Data was generated using this [script](https://github.com/JARVIS-Materials-Design/cdvae/blob/main/scripts/generate_data_cdvae.py).

In [7]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [8]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=supercon expname=supercon_test02 model.predict_property=True

/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/usr/local/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'default': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
[2024-04-06 22:19:47,574][hydra.utils][INFO] - Instantiating <cdvae.pl_data.datamodule.CrystDataModule>
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_b

In [9]:
!pip install smact

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.8 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


Export conda environment

In [10]:
!conda env export

name: base
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - archspec=0.2.2=pyhd8ed1ab_0
  - boltons=23.1.1=pyhd8ed1ab_0
  - brotli-python=1.1.0=py310hc6cd4ac_1
  - bzip2=1.0.8=hd590300_5
  - c-ares=1.24.0=hd590300_0
  - ca-certificates=2023.11.17=hbcca054_0
  - certifi=2023.11.17=pyhd8ed1ab_0
  - cffi=1.16.0=py310h2fee648_0
  - charset-normalizer=3.3.2=pyhd8ed1ab_0
  - colorama=0.4.6=pyhd8ed1ab_0
  - conda=23.11.0=py310hff52083_1
  - conda-libmamba-solver=23.12.0=pyhd8ed1ab_0
  - conda-package-handling=2.2.0=pyh38be061_0
  - conda-package-streaming=0.9.0=pyhd8ed1ab_0
  - distro=1.8.0=pyhd8ed1ab_0
  - fmt=10.1.1=h00ab1b0_1
  - icu=73.2=h59595ed_0
  - idna=3.6=pyhd8ed1ab_0
  - jsonpatch=1.33=pyhd8ed1ab_0
  - jsonpointer=2.4=py310hff52083_3
  - keyutils=1.6.1=h166bdaf_0
  - krb5=1.21.2=h659d440_0
  - ld_impl_linux-64=2.40=h41732ed_0
  - libarchive=3.7.2=h2aa1ff5_1
  - libcurl=8.5.0=hca28451_0
  - libedit=3.1.20191231=he28a2e2_2
  - 

In [ ]:
# from datetime import date
# d1 = today.strftime("%Y-%m-%d")

In [11]:
temp_dir=!ls /content/cdvae/HYDRA_JOBS/singlerun/

In [12]:
os.environ['TMP_DIR']=temp_dir[0]

In [13]:
!echo $TMP_DIR

2024-04-06


Adjust path accordingly

In [14]:
%%time
!python scripts/evaluate.py --n_step_each 5 --num_batches_to_samples 5 --batch_size 5 --model_path "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks opt gen recon

/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/scripts/eval_utils.py:56: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(model_path)):
/usr/local/lib/python3.10/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
/content/c

In [15]:
!pip install matminer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 22.6 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pymatgen
    Found existing installation: pymatgen 2022.4.26
    Uninstalling pymatgen-2022.4.26:
      Successfully uninstalled pymatgen-2022.4.26


In [16]:
%%time
!python scripts/compute_metrics.py --root_path  "/content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02" --tasks   gen recon

/content/cdvae/cdvae/pl_data/dataset.py:126: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/cdvae/pl_data/datamodule.py:142: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")
/content/cdvae/scripts/eval_utils.py:50: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(str(model_path)):
/usr/local/lib/python3.10/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  deprecation_warning(message=message)
  0% 0/105

In [17]:
temp_dir[0]

'2024-04-06'

In [18]:
!ls /content/cdvae/HYDRA_JOBS/singlerun/$TMP_DIR/supercon_test02/

'epoch=59-step=2040.ckpt'   eval_metrics.json   eval_recon.pt   lattice_scaler.pt   run.log
 eval_gen.pt		    eval_opt.pt         hparams.yaml    prop_scaler.pt


In [19]:
import torch
from jarvis.core.atoms import Atoms
from jarvis.core.atoms import pmg_to_atoms
from jarvis.core.lattice import Lattice
from jarvis.core.specie import atomic_numbers_to_symbols
from jarvis.db.jsonutils import dumpjson
from jarvis.analysis.structure.spacegroup import Spacegroup3D
from collections import Counter
from pymatgen.core.structure import Structure
import pandas as pd
opt_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_opt.pt"
gen_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_gen.pt"
recon_path = "/content/cdvae/HYDRA_JOBS/singlerun/"+temp_dir[0]+"/supercon_test02/eval_recon.pt"
csv_path = "/content/cdvae/data/supercon/test.csv"
df = pd.read_csv(csv_path)
x = torch.load(recon_path)
y = torch.load(gen_path)
z = torch.load(opt_path)
print(len(df),x["num_atoms"].shape,y["num_atoms"].shape,z["num_atoms"].shape)

105 torch.Size([1, 105]) torch.Size([1, 25]) torch.Size([1, 275])


In [20]:
num_atoms = x["num_atoms"]
atom_types = x["atom_types"]
frac_coords = x["frac_coords"]
lengths = x["lengths"]
angles = x["angles"]
index_list = torch.cumsum(num_atoms[0], dim=0).numpy().tolist()
indice_tuples = []
for i, ii in enumerate(index_list):
    if i == 0:
        tup = [0, index_list[i] - 1]
    else:
        tup = [index_list[i - 1] - 1, index_list[i] - 1]
    indice_tuples.append(tup)

recon_structures = []

for id_needed in range(num_atoms.shape[1]):
    id_fracs = frac_coords[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_atom_types = atom_types[0].numpy()[
        indice_tuples[id_needed][0] : indice_tuples[id_needed][1]
    ]
    id_lengths = lengths[0].numpy()[id_needed]
    id_angles = angles[0].numpy()[id_needed]
    lat = Lattice.from_parameters(
        id_lengths[0],
        id_lengths[1],
        id_lengths[2],
        id_angles[0],
        id_angles[1],
        id_angles[2],
    ).matrix
    atoms = Atoms(
        lattice_mat=lat,
        elements=atomic_numbers_to_symbols(id_atom_types),
        coords=id_fracs,
        cartesian=False,
    )
    # spg_numb = Spacegroup3D(atoms).space_group_number
    # spg_numbs.append(spg_numb)

    # print()
    # print()
    # print()
    # print("jarvis\n", atoms)
    # struct = Structure(
    #    lattice=Lat.from_parameters(
    #        id_lengths[0],
    #        id_lengths[1],
    #        id_lengths[2],
    #        id_angles[0],
    #        id_angles[1],
    #        id_angles[2],
    #    ),
    #    species=id_atom_types,
    #    coords=id_fracs,
    #    coords_are_cartesian=False,
    # )
    # atoms = pmg_to_atoms(struct)
    # print("pmg\n", atoms)
    # print()
    # print()
    # print()

    # gen_structures.append(atoms.to_dict())
    recon_structures.append(atoms)


In [21]:
test_structures=[]
jids = []
for i,ii in df.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  test_structures.append(atoms)#.to_dict())
  jids.append(ii['material_id'])

/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 16 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 2 fractional coo

In [22]:
df

,Unnamed: 0,material_id,cif,prop
0,0,JVASP-68993,# generated using pymatgen\ndata_Ba2LiSn\n_sym...,-1.191012
1,1,JVASP-134327,# generated using pymatgen\ndata_CaCO3\n_symme...,-7.781567
2,2,JVASP-102572,# generated using pymatgen\ndata_Ti3Si\n_symme...,-10.491911
3,3,JVASP-16365,# generated using pymatgen\ndata_LiAl3\n_symme...,-0.816908
4,4,JVASP-36657,# generated using pymatgen\ndata_VN\n_symmetry...,-2.649513
...,...,...,...,...
100,100,JVASP-67276,# generated using pymatgen\ndata_Be2NiPt\n_sym...,-0.478394
101,101,JVASP-139460,# generated using pymatgen\ndata_MgTe2\n_symme...,-3.793655
102,102,JVASP-20204,# generated using pymatgen\ndata_Ti3Al\n_symme...,-3.720508
103,103,JVASP-69411,# generated using pymatgen\ndata_Be2CuRh\n_sym...,-1.779018


Uploading to JARVIS-Leaderboard.

In [23]:
from jarvis.io.vasp.inputs import Poscar
import json
f=open('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv','w')
line='id,target,prediction\n'
f.write(line)
for i,j,k in zip(test_structures,recon_structures,jids):
  print(k,i.composition.reduced_formula,j.composition.reduced_formula)
  line = k+","+Poscar(i).to_string().replace('\n','\\n')+","+Poscar(j).to_string().replace('\n','\\n')+"\n"
  #line = k+","+json.dumps(i.to_dict())+","+json.dumps(j.to_dict())+"\n"
  f.write(line)
f.close()
#zip file before uploading to JARVIS-Leadrboard

JVASP-68993 Ba2LiSn Ba2Te
JVASP-134327 CaCO3 CuBO3
JVASP-102572 Ti3Si Ti5AlTeO
JVASP-16365 LiAl3 TiAl2Ir
JVASP-36657 VN AlN3
JVASP-20620 YB6 NbVB5
JVASP-15923 Ta2N YMgTa5B
JVASP-136988 MgP2 TaNiH
JVASP-13745 CaSi CaSi2P
JVASP-109989 Li3Sn Li2SiAu
JVASP-69869 Be2PtRh LiBePtRh
JVASP-129329 NbB BeNb
JVASP-78768 FeCo NbCo
JVASP-139524 YAlS3 NaCoS2N
JVASP-14781 YPd3 PPd3
JVASP-100039 AlVNi2 Ni2RuF
JVASP-16459 BeRh NiRh
JVASP-15255 SrAlSi Si2Rh
JVASP-19777 Cr3Ir AlCr6Ir
JVASP-20626 V3Pt NbVCrPt5
JVASP-101611 TaZn3 VZn3
JVASP-16521 ScGaNi2 ScZnNi2
JVASP-115500 B2SO2 NiBSO
JVASP-100364 NbMoO4 B2NO2F
JVASP-139627 NaTiO2 NaTiO2
JVASP-15075 HfN NbO
JVASP-35497 Y2Fe2Si2C NaNbFe2SiS
JVASP-72005 BeCo2P Be2SiB
JVASP-134821 POF BeF2
JVASP-135523 NaO2 NaSiSnO
JVASP-120382 AsP2 MoPO
JVASP-39656 TiNbRe2 Nb3P
JVASP-78464 TiAl TiNb
JVASP-37463 YNbRu2 MgNbAlRu
JVASP-18243 TiSi MgTi
JVASP-40528 LiCa2Ga CaTiGa2
JVASP-16432 YAl3 NbAl2Ga
JVASP-134358 SiCF2 NaBeAlF
JVASP-16775 HfBeSi HfSiAs
JVASP-18919 ScRh3 BeR

In [24]:
dfx = pd.read_csv('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv')

In [25]:
import pandas as pd
info = {}
test_path = pd.read_csv("/content/cdvae/data/supercon/test.csv")
val_path = pd.read_csv("/content/cdvae/data/supercon/val.csv")
train_path = pd.read_csv("/content/cdvae/data/supercon/train.csv")
test={}
val={}
train={}

for i,ii in train_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  train[jid]=pos

for i,ii in val_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  val[jid]=pos

for i,ii in test_path.iterrows():
  atoms=pmg_to_atoms(Structure.from_str(ii['cif'],fmt='cif'))
  pos = Poscar(atoms).to_string().replace('\n','\\n')
  jid=ii['material_id']
  test[jid]=pos
info['train']=train
info['val']=val
info['test']=test

/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 2 fractional coor

In [26]:
from jarvis.db.jsonutils import dumpjson
dumpjson(data=info,filename='dft_3d_Tc_supercon.json')

In [27]:
test_path

,Unnamed: 0,material_id,cif,prop
0,0,JVASP-68993,# generated using pymatgen\ndata_Ba2LiSn\n_sym...,-1.191012
1,1,JVASP-134327,# generated using pymatgen\ndata_CaCO3\n_symme...,-7.781567
2,2,JVASP-102572,# generated using pymatgen\ndata_Ti3Si\n_symme...,-10.491911
3,3,JVASP-16365,# generated using pymatgen\ndata_LiAl3\n_symme...,-0.816908
4,4,JVASP-36657,# generated using pymatgen\ndata_VN\n_symmetry...,-2.649513
...,...,...,...,...
100,100,JVASP-67276,# generated using pymatgen\ndata_Be2NiPt\n_sym...,-0.478394
101,101,JVASP-139460,# generated using pymatgen\ndata_MgTe2\n_symme...,-3.793655
102,102,JVASP-20204,# generated using pymatgen\ndata_Ti3Al\n_symme...,-3.720508
103,103,JVASP-69411,# generated using pymatgen\ndata_Be2CuRh\n_sym...,-1.779018


In [28]:
!cp scripts/compute_metrics.py scripts/eval_utils.py .

In [29]:
!ls

AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv  dft_3d_Tc_supercon.json  README.md
assets					     env.cpu.yml	      requirements.txt
cdvae					     env_sub.yml	      scripts
cdvae.egg-info				     env.yml		      setup.py
compute_metrics.py			     eval_utils.py	      WABDB
conf					     HYDRA_JOBS
data					     LICENSE


In [39]:
import numpy as np
from pymatgen.analysis.structure_matcher import StructureMatcher
import pandas as pd
from jarvis.io.vasp.inputs import Poscar
from tqdm import tqdm
df=pd.read_csv('AI-AtomGen-Tc_supercon-dft_3d-test-rmse.csv')

matcher = StructureMatcher(stol=0.5, angle_tol=10, ltol=0.3)
rms = []
for m, mm in tqdm(df.iterrows()):
    try:
        atoms_target = (
            Poscar.from_string(
                (mm["target"].replace("\\n", "\n"))
            ).atoms
        ).pymatgen_converter()
        atoms_pred = (
            Poscar.from_string(
                (mm["prediction"].replace("\\n", "\n"))
            ).atoms
        ).pymatgen_converter()
        # rms_dist = matcher.get_rms_dist(atoms_pred,atoms_target)
        rms_dist = matcher.get_rms_anonymous(atoms_pred, atoms_target)
        if rms_dist[0] is not None:
            rms.append(rms_dist[0])
    except Exception as exp:
        print("exp", exp)
        pass
rms = round(np.array(rms).mean(), 4)
print('rms', rms)

105it [00:03, 29.76it/s]

rms 0.3406


In [40]:
!conda env export

name: base
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - archspec=0.2.2=pyhd8ed1ab_0
  - boltons=23.1.1=pyhd8ed1ab_0
  - brotli-python=1.1.0=py310hc6cd4ac_1
  - bzip2=1.0.8=hd590300_5
  - c-ares=1.24.0=hd590300_0
  - ca-certificates=2023.11.17=hbcca054_0
  - certifi=2023.11.17=pyhd8ed1ab_0
  - cffi=1.16.0=py310h2fee648_0
  - charset-normalizer=3.3.2=pyhd8ed1ab_0
  - colorama=0.4.6=pyhd8ed1ab_0
  - conda=23.11.0=py310hff52083_1
  - conda-libmamba-solver=23.12.0=pyhd8ed1ab_0
  - conda-package-handling=2.2.0=pyh38be061_0
  - conda-package-streaming=0.9.0=pyhd8ed1ab_0
  - distro=1.8.0=pyhd8ed1ab_0
  - fmt=10.1.1=h00ab1b0_1
  - icu=73.2=h59595ed_0
  - idna=3.6=pyhd8ed1ab_0
  - jsonpatch=1.33=pyhd8ed1ab_0
  - jsonpointer=2.4=py310hff52083_3
  - keyutils=1.6.1=h166bdaf_0
  - krb5=1.21.2=h659d440_0
  - ld_impl_linux-64=2.40=h41732ed_0
  - libarchive=3.7.2=h2aa1ff5_1
  - libcurl=8.5.0=hca28451_0
  - libedit=3.1.20191231=he28a2e2_2
  - 

In [47]:
!pip install matminer==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 76.7 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1
  Attempting uninstall: matminer
    Found existing installation: matminer 0.9.2
    Uninstalling matminer-0.9.2:
      Successfully uninstalled matminer-0.9.2


In [68]:
!rm -rf /usr/local/lib/python3.10/dist-packages/pandas*
!pip uninstall pandas -y
!pip install pandas==1.5.3

Found existing installation: pandas 1.5.3
Uninstalling pandas-1.5.3:
  Successfully uninstalled pandas-1.5.3
  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
from p_tqdm import p_map
from compute_metrics import GenEval,get_crystal_array_list,Crystal, RecEval
crys_array_list, true_crystal_array_list = get_crystal_array_list(recon_path)
pred_crys = p_map(lambda x: Crystal(x), crys_array_list)
gt_crys = p_map(lambda x: Crystal(x), true_crystal_array_list)
rec_evaluator = RecEval(pred_crys, gt_crys)
recon_metrics = rec_evaluator.get_metrics()


crys_array_list, _ = get_crystal_array_list(gen_path)
gen_crys = p_map(lambda x: Crystal(x), crys_array_list)
gen_evaluator = GenEval(gen_crys, gt_crys, eval_model_name='carbon')
gen_metrics = gen_evaluator.get_metrics()
print(recon_metrics)

ModuleNotFoundError: No module named 'compute_metrics'

In [31]:
print(recon_metrics)

NameError: name 'recon_metrics' is not defined

In [ ]:
print(gen_metrics)

In [ ]:
pip install alignn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.0
    Uninstalling pyparsing-3.1.0:
      Successfully uninstalled pyparsing-3.1.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. T

In [ ]:
from alignn.pretrained import get_multiple_predictions


In [ ]:
atoms

CaGaO3
1.0
3.83708195 0.0 2.3495350640667923e-16
6.170498392460395e-16 3.83708195 2.3495350640667923e-16
0.0 0.0 3.83708195
Ca Ga O
1 1 3
Cartesian
0.0 0.0 0.0
1.91854095927 1.91854095927 1.91854095927
1.91854095927 1.91854095927 0.0
1.91854095927 0.0 1.91854095927
0.0 1.91854095927 1.91854095927

In [ ]:
atoms.write_poscar('POSCAR')

Quickly predict Tc and other properties

In [ ]:
!pretrained.py --model_name jv_supercon_tc_alignn --file_format poscar --file_path POSCAR

100% 44.5M/44.5M [00:03<00:00, 13.0MiB/s]
Using chk file jv_supercon_tc_alignn/checkpoint_200.pt from  ['jv_supercon_tc_alignn/checkpoint_200.pt']
Path /usr/local/bin/jv_supercon_tc_alignn.zip
Config /content/cdvae/jv_supercon_tc_alignn/config.json
Predicted value: jv_supercon_tc_alignn POSCAR [6.236893177032471]


In [ ]:
!pretrained.py --model_name jv_supercon_debye_alignn --file_format poscar --file_path POSCAR

100% 45.0M/45.0M [00:03<00:00, 12.4MiB/s]
Using chk file jv_supercon_debye_alignn/checkpoint_300.pt from  ['jv_supercon_debye_alignn/checkpoint_300.pt']
Path /usr/local/bin/jv_supercon_debye_alignn.zip
Config /content/cdvae/jv_supercon_debye_alignn/config.json
Predicted value: jv_supercon_debye_alignn POSCAR [519.1744384765625]


In [ ]:
!pretrained.py --model_name jv_supercon_edos_alignn --file_format poscar --file_path POSCAR

100% 45.0M/45.0M [00:03<00:00, 13.1MiB/s]
Using chk file jv_supercon_edos_alignn/checkpoint_500.pt from  ['jv_supercon_edos_alignn/checkpoint_500.pt']
Path /usr/local/bin/jv_supercon_edos_alignn.zip
Config /content/cdvae/jv_supercon_edos_alignn/config.json
Predicted value: jv_supercon_edos_alignn POSCAR [3.9772684574127197]


In [ ]:
!pretrained.py --model_name jv_supercon_a2F_alignn --file_format poscar --file_path POSCAR

100% 44.7M/44.7M [00:03<00:00, 12.9MiB/s]
Using chk file jv_supercon_a2F_alignn/checkpoint_200.pt from  ['jv_supercon_a2F_alignn/checkpoint_200.pt']
Path /usr/local/bin/jv_supercon_a2F_alignn.zip
Config /content/cdvae/jv_supercon_a2F_alignn/config.json
Predicted value: jv_supercon_a2F_alignn POSCAR [-0.04695920646190643, -0.08268657326698303, 0.0062256790697574615, 0.024442754685878754, -0.12895868718624115, 0.032128673046827316, -0.09102485328912735, 0.02549310401082039, 0.03703967481851578, 0.0009269975125789642, -0.030133794993162155, 0.15729741752147675, -0.006535707041621208, 0.1515427529811859, 0.01641899347305298, 0.03273272514343262, 0.02994958683848381, 0.13513663411140442, 0.17497950792312622, 0.07090292870998383, 0.16760031878948212, 0.007767858449369669, -0.06977668404579163, 0.13795417547225952, -0.048686932772397995, 0.030890438705682755, 0.15939219295978546, 0.042231615632772446, 0.10495894402265549, 0.09189825505018234, -0.07477183640003204, -0.04807336628437042, -0.101

In [ ]:
!pretrained.py --model_name jv_formation_energy_peratom_alignn --file_format poscar --file_path POSCAR

100% 47.5M/47.5M [00:03<00:00, 13.2MiB/s]
Using chk file jv_formation_energy_peratom_alignn/checkpoint_300.pt from  ['jv_formation_energy_peratom_alignn/checkpoint_300.pt']
Path /usr/local/bin/jv_formation_energy_peratom_alignn.zip
Config /content/cdvae/jv_formation_energy_peratom_alignn/config.json
Predicted value: jv_formation_energy_peratom_alignn POSCAR [-2.161196708679199]


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.21.0
aiohttp==3.8.5
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
antlr4-python3-runtime==4.9.3
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.4.0
arviz==0.15.1
ase==3.22.1
astropy==5.2.2
astunparse==1.6.3
async-timeout==4.0.2
attrs==23.1.0
audioread==3.0.0
autograd==1.6.2
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blinker==1.4
blis==0.7.10
blosc2==2.0.0
bokeh==3.1.1
branca==0.6.0
build==0.10.0
CacheControl==0.13.1
cachetools==5.3.1
catalogue==2.0.9
-e git+https://github.com/JARVIS-Materials-Design/cdvae.git@ec669e7cf8ccc90b92abc88793243cbb4d0e58a2#egg=cdvae
certifi==2023.7.22
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.6
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.1.0
cons==0.4.6
contextlib2==21.6.0
contourpy==1.1.0
convertd